<a href="https://colab.research.google.com/github/peterliu502/GEO1001_hw02/blob/master/5386586_5360684.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analysis report for GEO1001 HW01 
---

## Authors  
1. First author: [<img src="https://avatars3.githubusercontent.com/u/59593272?s=400&u=ba1618be6d5e354f0bd7685ff405bdec6d18c101&v=4" align = "left" width = "25" height = "25" />](https://github.com/peterliu502)  
   * Name: Zhenyu Liu  
   * Student Number: 5386586  
  
2. Second author: [<img src="https://avatars2.githubusercontent.com/u/47234206?s=400&u=3f54e18f68e48f985db9f0ef1a8eb3a3ca189b1e&v=4" align = "left" style = "float:left" width = "25" height = "25" />](https://github.com/Ziyan-Wu)
  * Name: Ziyan Wu
  * Student Number: 5360684

## Source data 
The source data contains `Sentinel-2` images for the area around Delft on May 30th 2020, with 10m, 20m and 60m resolutions.

## Python packages  
This assignment uses following Python packages:  
1. `numpy`
2. `scikit-learn ` 
3. `matplotlib`  
4. `rasterio` 

In [ ]:
import numpy as np
import rasterio as rio
from rasterio.windows import Window
import rasterio.plot as rp
from matplotlib import pyplot as plt
from matplotlib import colors as mc
from sklearn import cluster

## Open files and preprocess  

### Open 60m data  
Open and read `band02`, `band03`, `band04` and `band8A` of the raster data in 60m resolution.

In [ ]:
# open the T31UET_20200530T105031_B2_60m.jp2
with rio.open(
        './GRANULE/L2A_T31UET_A025788_20200530T105134/IMG_DATA/R60m/T31UET_20200530T105031_B02_60m.jp2',
        driver="JP2OpenJPEG") as ds_60_B2:
    # Cut the raster and standardize the raster values
    ds_60_B2_subset = rp.adjust_band(ds_60_B2.read(1))

    # open the T31UET_20200530T105031_B3_60m.jp2
    with rio.open(
            './GRANULE/L2A_T31UET_A025788_20200530T105134/IMG_DATA/R60m/T31UET_20200530T105031_B03_60m.jp2',
            driver="JP2OpenJPEG") as ds_60_B3:
        # get the row and column number for the top-left pixel
        row2, col2 = ds_60_B3.index(601200, 5773695)
        # standardize the raster values
        ds_60_B3_subset = rp.adjust_band(ds_60_B3.read(1))

        # open the T31UET_20200530T105031_B4_60m.jp2
        with rio.open(
                './GRANULE/L2A_T31UET_A025788_20200530T105134/IMG_DATA/R60m/T31UET_20200530T105031_B04_60m.jp2',
                driver="JP2OpenJPEG") as ds_60_B4:
            # get the row and column number for the top-left pixel
            row3, col3 = ds_60_B4.index(601200, 5773695)
            # standardize the raster values
            ds_60_B4_subset = rp.adjust_band(ds_60_B4.read(1))

            # open the T31UET_20200530T105031_B8_60m.jp2
            with rio.open(
                    './GRANULE/L2A_T31UET_A025788_20200530T105134/IMG_DATA/R60m/T31UET_20200530T105031_B8A_60m.jp2',
                    driver="JP2OpenJPEG") as ds_60_B8:
                # standardize the raster values
                ds_60_B8_subset = rp.adjust_band(ds_60_B8.read(1))
                # merge the bands 4, 3 and 2 (true color)
                ds_60_432 = np.dstack((ds_60_B4_subset, ds_60_B3_subset, ds_60_B2_subset))
                # merge the bands 8, 4 and 3 (nir false color)
                ds_60_843 = np.dstack((ds_60_B8_subset, ds_60_B4_subset, ds_60_B3_subset))
                ds_60_B8.close()
            ds_60_B4.close()
        ds_60_B3.close()
    ds_60_B2.close()

#### Open 10m data   
Open and read `band02`, `band03`, `band04` and `band8` of the raster data in 10m resolution.

In [ ]:
# open the T31UET_20200530T105031_B02_10m.jp2
with rio.open(
        './GRANULE/L2A_T31UET_A025788_20200530T105134/IMG_DATA/R10m/T31UET_20200530T105031_B02_10m.jp2',
        driver="JP2OpenJPEG") as ds_10_B02:
    # get the row and column number for the top-left pixel
    row1, col1 = ds_10_B02.index(601200, 5773695)
    # cut the raster and standardize the raster values
    ds_10_B02_subset = rp.adjust_band(ds_10_B02.read(1, window=Window(col1, row1, 700, 500)))

    # open the T31UET_20200530T105031_B03_10m.jp2
    with rio.open(
            './GRANULE/L2A_T31UET_A025788_20200530T105134/IMG_DATA/R10m/T31UET_20200530T105031_B03_10m.jp2',
            driver="JP2OpenJPEG") as ds_10_B03:
        # get the row and column number for the top-left pixel
        row2, col2 = ds_10_B03.index(601200, 5773695)
        # cut the raster and standardize the raster values
        ds_10_B03_subset = rp.adjust_band(ds_10_B03.read(1, window=Window(col2, row2, 700, 500)))

        # open the T31UET_20200530T105031_B04_10m.jp2
        with rio.open(
                './GRANULE/L2A_T31UET_A025788_20200530T105134/IMG_DATA/R10m/T31UET_20200530T105031_B04_10m.jp2',
                driver="JP2OpenJPEG") as ds_10_B04:
            # get the row and column number for the top-left pixel
            row3, col3 = ds_10_B04.index(601200, 5773695)
            # cut the raster and standardize the raster values
            ds_10_B04_subset = rp.adjust_band(ds_10_B04.read(1, window=Window(col3, row3, 700, 500)))

            # open the T31UET_20200530T105031_B08_10m.jp2
            with rio.open(
                    './GRANULE/L2A_T31UET_A025788_20200530T105134/IMG_DATA/R10m/T31UET_20200530T105031_B08_10m.jp2',
                    driver="JP2OpenJPEG") as ds_10_B08:
                # get the row and column number for the top-left pixel
                row3, col3 = ds_10_B04.index(601200, 5773695)
                # cut the raster and standardize the raster values
                ds_10_B08_subset = rp.adjust_band(ds_10_B08.read(1, window=Window(col3, row3, 700, 500)))
                # merge the bands 4, 3 and 2 (true color)
                ds_10_432 = np.dstack((ds_10_B04_subset, ds_10_B03_subset, ds_10_B02_subset))
                # merge the bands 8, 4 and 3 (nir false color)
                ds_10_843 = np.dstack((ds_10_B08_subset, ds_10_B04_subset, ds_10_B03_subset))
                ds_10_B08.close()
            ds_10_B04.close()
        ds_10_B03.close()
    ds_10_B02.close()

## KMeans
KMeans is a commonly used classification method. Its basic idea is assigning points to K clusters so that each point is nearest to its `cluster center` (`cluster mean`) than other cluster centers.  

One of the key points of KMeans clustering is to find the optimal `K-value` (`n_clusters`) in advance. In this assignment, we use `SSE` (`Error Sum of Squares`) to find the `optimal K-value`. The main idea of this solution is that the value of SSE will decrease with the increase of K-value in general, but if the K-value is less than the optimal K-value, the SSE will decrease sharply. After K-value is greater than the optimal K-value, SSE decreases very slowly. Therefore, the goal is to find the turning point on the SSE curve, which is the optimal K-value position. In the following sections, we will verify the optimal K-value with the output classification images.

In [ ]:
def sse(arr):
    # store the SSE of each result
    SSE = []  
    for k in range(3, 11):
        # create a KMeans classifier object
        estimator = cluster.KMeans(n_clusters=k)
        estimator.fit(arr)
        SSE.append(estimator.inertia_)
    plt.figure(figsize=(5, 5))
    X = range(3, 11)
    plt.xlabel('k')
    plt.ylabel('SSE')
    plt.plot(X, SSE, 'o-')
    plt.show()

### Plot 60m data


#### Find optimal K-value of true color classification images
According to the SSE curve, the optimal K-value is 6.

In [ ]:
# create an empty array with same dimension and data type
ds_60_432_1d = ds_60_432[:, :, :3].reshape((ds_60_432.shape[0] * ds_60_432.shape[1], ds_60_432.shape[2]))
# plot a SSE curve
sse(ds_60_432_1d)

#### Plot true color classification images


In [ ]:
# plot true color classification image in 60m resolution
ds_60_fig432 = plt.figure(figsize=(30, 15))
# output the classification image with K-value equals 3 to 10 in turn.
for elm432 in range(11)[3:]:
    # create a KMeans classifier object
    ds_60_cl = cluster.KMeans(n_clusters=elm432)
    # train the data
    ds_60_param = ds_60_cl.fit(ds_60_432_1d)
    # get the labels of the classes
    ds_60_img_cl = ds_60_cl.labels_
    # reshape labels to a 3d array (one band only)
    ds_60_img_cl = ds_60_img_cl.reshape(ds_60_432[:, :, 0].shape)
    # plot the classification image
    ds_60_fig432.add_subplot(2, 4, elm432 - 2)
    # set the custom color map to represent the different classes in image
    cmap = mc.LinearSegmentedColormap.from_list(
        "", ["seagreen", "tan", "white", "green", "mediumseagreen", "yellow",
             "magenta", "red", "blue"])
    plt.imshow(ds_60_img_cl, cmap=cmap)
    plt.title("n_clusters="+str(elm432))

plt.show()

#### Find optimal K-value of nir false color classification images
According to the SSE curve, the optimal K-value is 6.

In [ ]:
# create an empty array with same dimension and data type
ds_60_843_1d = ds_60_843[:, :, :3].reshape((ds_60_843.shape[0] * ds_60_843.shape[1], ds_60_843.shape[2]))
# plot a SSE curve
sse(ds_60_843_1d)

#### Plot nir false color classification images

In [ ]:
# plot nir false color classification image in 60m resolution
ds_60_fig843 = plt.figure(figsize=(30, 15))
# output the classification image with K-value equals 3 to 10 in turn.
for elm843 in range(11)[3:]:
    # create a KMeans classifier object
    ds_60_cl = cluster.KMeans(n_clusters=elm843)
    # train the data
    ds_60_param = ds_60_cl.fit(ds_60_843_1d)
    # get the labels of the classes
    ds_60_img_cl = ds_60_cl.labels_
    # reshape labels to a 3d array (one band only)
    ds_60_img_cl = ds_60_img_cl.reshape(ds_60_843[:, :, 0].shape)
    # plot the classification image
    ds_60_fig843.add_subplot(2, 4, elm843 - 2)
    # set the custom color map to represent the different classes in image
    cmap = mc.LinearSegmentedColormap.from_list(
        "", ["seagreen", "tan", "white", "green", "mediumseagreen", "yellow",
             "magenta", "red", "blue"])
    plt.imshow(ds_60_img_cl, cmap=cmap)
    plt.title("n_clusters="+str(elm843))

plt.show()

### Plot 10m data

#### Find optimal K-value of true color classification images
According to the SSE curve, the optimal K-value is 6.

In [ ]:
# create an empty array with same dimension and data type
ds_10_432_1d = ds_10_432[:, :, :3].reshape((ds_10_432.shape[0] * ds_10_432.shape[1], ds_10_432.shape[2]))
# plot a SSE curve
sse(ds_10_432_1d)

#### Plot true color classification images


In [ ]:
# plot true color classification image in 10m resolution
ds_10_fig432 = plt.figure(figsize=(30, 10))
# output the classification image with K-value equals 3 to 10 in turn.
for elm432 in range(11)[3:]:
    # create a KMeans classifier object
    ds_10_cl = cluster.KMeans(n_clusters=elm432)
    # train the data
    ds_10_param = ds_10_cl.fit(ds_10_432_1d)
    # get the labels of the classes
    ds_10_img_cl = ds_10_cl.labels_
    # reshape labels to a 3d array (one band only)
    ds_10_img_cl = ds_10_img_cl.reshape(ds_10_432[:, :, 0].shape)
    # plot the classification image
    ds_10_fig432.add_subplot(2, 4, elm432 - 2)
    # set the custom color map to represent the different classes in image
    cmap = mc.LinearSegmentedColormap.from_list(
        "", ["seagreen", "tan", "white", "green", "mediumseagreen", "yellow",
             "magenta", "red", "blue"])
    plt.imshow(ds_10_img_cl, cmap=cmap)
    plt.title("n_clusters="+str(elm432))

plt.show()

#### Find optimal K-value of nir false color classification images
According to the SSE curve, the optimal K-value is 6.

In [ ]:
# create an empty array with same dimension and data type
ds_10_843_1d = ds_10_843[:, :, :3].reshape((ds_10_843.shape[0] * ds_10_843.shape[1], ds_10_843.shape[2]))
# plot a SSE curve
sse(ds_10_843_1d)

#### Plot nir false color classification images

In [ ]:
# plot nir false color classification image in 10m resolution
ds_10_fig843 = plt.figure(figsize=(30, 10))
# output the classification image with K-value equals 3 to 10 in turn.
for elm843 in range(11)[3:]:
    # create a KMeans classifier object
    ds_10_cl = cluster.KMeans(n_clusters=elm843)
    # train the data
    ds_10_param = ds_10_cl.fit(ds_10_843_1d)
    # get the labels of the classes
    ds_10_img_cl = ds_10_cl.labels_
    # reshape labels to a 3d array (one band only)
    ds_10_img_cl = ds_10_img_cl.reshape(ds_10_843[:, :, 0].shape)
    # plot the classification image
    ds_10_fig843.add_subplot(2, 4, elm843 - 2)
    # set the custom color map to represent the different classes in image
    cmap = mc.LinearSegmentedColormap.from_list(
        "", ["seagreen", "tan", "white", "green", "mediumseagreen", "yellow",
             "magenta", "red", "blue"])
    plt.imshow(ds_10_img_cl, cmap=cmap)
    plt.title("n_clusters="+str(elm843))

plt.show()